<a href="https://colab.research.google.com/github/saumyea/BDA-Project/blob/main/GUI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import scipy.sparse

**Importing the .NPZ files**

In [ ]:
# Define file paths in your Google Drive for loading the matrices
user_similarity_load_path = '/content/drive/MyDrive/sem7/BDA/MP-mtx_files/user_similarity_matrix.npz'
item_similarity_load_path = '/content/drive/MyDrive/sem7/BDA/MP-mtx_files/item_similarity_matrix.npz'
sparse_matrix_load_path = '/content/drive/MyDrive/sem7/BDA/MP-mtx_files/sparse_matrix.npz'
# Load the matrices from .npz files
user_similarity_matrix = scipy.sparse.load_npz(user_similarity_load_path)
item_similarity_matrix = scipy.sparse.load_npz(item_similarity_load_path)
sparse_matrix = scipy.sparse.load_npz(sparse_matrix_load_path)

In [ ]:
print(type(user_similarity_matrix))  # Should be a sparse matrix
print(type(item_similarity_matrix))
print(type(sparse_matrix))

<class 'scipy.sparse._csr.csr_matrix'>
<class 'scipy.sparse._csr.csr_matrix'>
<class 'scipy.sparse._csr.csr_matrix'>


**Importing The Dictionaries**

In [ ]:
import joblib

# Specify the same path in your Google Drive
load_path = '/content/drive/MyDrive/sem7/BDA/MP-mtx_files/'

# Load the dictionaries from the .pkl files
user_id_to_index = joblib.load(load_path + 'user_id_to_index.pkl')
index_to_title = joblib.load(load_path + 'index_to_title.pkl')

**Gradio for GUI**

In [ ]:
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.7/56.7 MB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.8/319.8 kB 23.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.0/11.0 MB 94.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.3/73.3 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 10.2 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2


In [ ]:
import gradio as gr

# Collaborative filtering
def user_based(user_sim_matrix, ratings_matrix, user_idx, num_similar_users, num_recommendations):
    # Get indices of the most similar users for the specified user
    similar_users = np.argsort(-user_sim_matrix[user_idx])[:num_similar_users]

    # Initialize a dictionary to store aggregated recommendation scores
    item_scores = {}

    for sim_user_idx in similar_users:
        # Get the ratings of the similar user
        user_ratings = ratings_matrix[sim_user_idx].toarray()[0]

        # Update scores for items that the target user hasn't rated yet
        for item_idx, rating in enumerate(user_ratings):
            if ratings_matrix[user_idx, item_idx] == 0:  # Item not rated by target user
                if item_idx not in item_scores:
                    item_scores[item_idx] = 0
                item_scores[item_idx] += rating * user_sim_matrix[user_idx, sim_user_idx]

    # Sort items by aggregated scores and return the top N items
    sorted_items = sorted(item_scores.items(), key=lambda x: x[1], reverse=True)
    return [item[0] for item in sorted_items[:num_recommendations]]

def item_based(item_sim_matrix, ratings_matrix, user_idx, num_recommendations):
    # Get the indices of items rated by the target user
    rated_item_indices = ratings_matrix[user_idx].nonzero()[1]

    # Initialize a dictionary to hold aggregated recommendation scores
    item_scores = {}

    for item_idx in rated_item_indices:
        # Get similarity scores for the rated item
        similar_item_indices = np.argsort(-item_sim_matrix[item_idx].toarray().flatten())

        for sim_item_idx in similar_item_indices:
            if sim_item_idx not in rated_item_indices:  # Not rated by the target user
                if sim_item_idx not in item_scores:
                    item_scores[sim_item_idx] = 0
                # Aggregate scores based on the user's rating and item similarity
                item_scores[sim_item_idx] += ratings_matrix[user_idx, item_idx] * item_sim_matrix[item_idx, sim_item_idx]

    # Sort items based on aggregated scores and return the top N items
    sorted_items = sorted(item_scores.items(), key=lambda x: x[1], reverse=True)
    return [item[0] for item in sorted_items[:num_recommendations]]

# GUI functions to handle predictions
def user_based_predict(user_id, num_similar_users, num_recommendations):
    user_id = int(user_id)
    num_similar_users = int(num_similar_users)
    num_recommendations = int(num_recommendations)

    print(f"User ID: {user_id}, Similar Users: {num_similar_users}, Recommendations: {num_recommendations}")

    if user_id in user_id_to_index:
        user_idx = user_id_to_index[user_id]
        recommendations = user_based(user_similarity_matrix, sparse_matrix, user_idx, num_similar_users, num_recommendations)
        print(f"Recommendations (indexes): {recommendations}")

        # Convert movie indices to titles and join them line by line
        return "\n".join([index_to_title[item] for item in recommendations])
    else:
        return "Invalid User ID"

def item_based_predict(user_id, num_recommendations):
    user_id = int(user_id)
    num_recommendations = int(num_recommendations)

    print(f"User ID: {user_id}, Recommendations: {num_recommendations}")

    if user_id in user_id_to_index:
        user_idx = user_id_to_index[user_id]
        recommendations = item_based(item_similarity_matrix, sparse_matrix, user_idx, num_recommendations)
        print(f"Recommendations (indexes): {recommendations}")

        # Convert movie indices to titles and join them line by line
        return "\n".join([index_to_title[item] for item in recommendations])
    else:
        return "Invalid User ID"

# Gradio interface definition with CSS for font size increase and custom heading color
with gr.Blocks(css="""
    .gradio-container { font-size: 20px; }
    h2 { color: #4CAF50; text-align: center;}
    h1 { color: #4CAF50; text-align: center;}
    label { font-size: 20px}
""") as demo:
    gr.Markdown("# Movie Recommendation System")
    with gr.Row():
        with gr.Column():
            gr.Markdown("## User-Based Collaborative Filtering")
            user_id_input = gr.Number(label="Enter User ID", value=10)
            num_similar_users_input = gr.Number(label="Number of Similar Users", value=5)
            num_recommendations_input = gr.Number(label="Number of Recommendations", value=10)
            user_based_output = gr.Textbox(label="User-Based Recommendations", lines=10)

            user_based_btn = gr.Button("Get User-Based Recommendations")
            user_based_btn.click(user_based_predict, inputs=[user_id_input, num_similar_users_input, num_recommendations_input], outputs=user_based_output)

        with gr.Column():
            gr.Markdown("## Item-Based Collaborative Filtering")
            user_id_input_item = gr.Number(label="Enter User ID", value=10)
            num_recommendations_input_item = gr.Number(label="Number of Recommendations", value=5)
            item_based_output = gr.Textbox(label="Item-Based Recommendations", lines=10)

            item_based_btn = gr.Button("Get Item-Based Recommendations")
            item_based_btn.click(item_based_predict, inputs=[user_id_input_item, num_recommendations_input_item], outputs=item_based_output)

# Launch the Gradio app
demo.launch()


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://2453c6cba430f0b5b0.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
